# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

device = torch.device("mps")


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [ ]:
from torch import nn
from math import sqrt

# 셀프 어텐션
class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim # 입력 차원  (보통 어휘 크기((vocab_size)))
    self.d_model = d_model     # 임베딩 차원 (모델 크기에 따라 다르다. BERT는 768..)

    self.wq = nn.Linear(input_dim, d_model)  # 쿼리(입력 -> d_model 차원 변환)
    self.wk = nn.Linear(input_dim, d_model)  #   키(입력 -> d_model 차원 변환)
    self.wv = nn.Linear(input_dim, d_model)  #   값(입력 -> d_model 차원 변환)

    # 최종 출력을 위한 선형 레이어 (d_model -> d_model)
    self.dense = nn.Linear(d_model, d_model)

    # 소프트맥스 (어텐션 가중치 계산을 위해((마지막 차원 기준으로 소프트맥스 수행)))
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    ### 1. Q, K, V 벡터 생성 – 입력을 통해 Wq, Wk, Wv를 선형 변환하여 Q, K, V를 만듬
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
	# Q (Query): 현재 기준이 되는 토큰 (검색 기준)
    # K (Key): 모든 토큰 (기준과 비교할 대상)
    # V (Value): 최종적으로 가중치를 적용해 반환할 값
    # 어텐션 매커니즘은 Q와 K의 유사도를 계산하여 V에서 어떤 정보를 얼마만큼 가져올지 결정한다.



    ### 2. 어텐션 스코어 계산 – 쿼리(Q)와 키(K)를 내적하여 유사도 계산 (Q * K^T)
    # W_q와 W_k는 문맥적으로 유사한 단어를 높은 내적 값을 내뱉도록 학습뇌다.
    # B(Batch Size: 배치 크기), S(Sequence length: 시퀀스 길이), D(Embedding Dimension: 임베딩 차원)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    # 에너지라고도 한다.
    # - 에너지 값이 크면: 쿼리와 키가 매우 유사하다는 의미
    # - 에너지 값이 작을스록 두 벡터간의 상관성이 낮다는 의미

    ### 3. 스코어 정규화 – sqrt(d_model)로 나눠서 스케일링 (값이 너무 크거나 작아지는 문제 방지(vanishing/exploding))
    score = score / sqrt(self.d_model)

    ### 4. 마스킹 – 필요 시 패딩이나 특정 토큰을 제외하기 위해 마스크를 적용
    if mask is not None:
      score = score + (mask * -1e9) # (마스크된 위치(값 1))에 매우 큰 음수 추가 -> 소프트 맥스에서 0이됨)

    ### 5. 소프트맥스를 사용해 어텐션 가중치 계산
    score = self.softmax(score)

	### 6. 어텐션 가중치 적용 – 값 벡터(V)에 어텐션 가중치를 곱해 최종 결과 생성
    # Wv는 유사도가 높은 단어로부터 '올바른 정보'를 가져오도록 학습된다.
    result = torch.matmul(score, v) # 가중치 합이 적용되어 컥텍스트 벡터가 된다.
    # 컨텍스트 벡터는 각 단어(밸류 벡터)에 어텐션 스코어를 곱한 가중합입니다.
    # C_3 = A_<3, 1>V3 + A_<3, 2>V2 + A<3, 3>V3 + A<3, 4>V4


    ### 7. 출력 변환 – 선형 변환(dense)을 통해 최종 출력 생성
    result = self.dense(result)


    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [ ]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 228.05881744623184
=========> Train acc: 0.770 | Test acc: 0.745
Epoch   1 | Train Loss: 181.79184779524803
=========> Train acc: 0.818 | Test acc: 0.776
Epoch   2 | Train Loss: 161.0439212024212
=========> Train acc: 0.847 | Test acc: 0.786
Epoch   3 | Train Loss: 144.77784506976604
=========> Train acc: 0.866 | Test acc: 0.798
Epoch   4 | Train Loss: 127.65684458613396
=========> Train acc: 0.892 | Test acc: 0.800
Epoch   5 | Train Loss: 113.74214302748442
=========> Train acc: 0.907 | Test acc: 0.804
Epoch   6 | Train Loss: 101.79052671045065
=========> Train acc: 0.916 | Test acc: 0.799
Epoch   7 | Train Loss: 90.75403520092368
=========> Train acc: 0.937 | Test acc: 0.809
Epoch   8 | Train Loss: 77.53205958008766
=========> Train acc: 0.945 | Test acc: 0.808
Epoch   9 | Train Loss: 68.24311332032084
=========> Train acc: 0.957 | Test acc: 0.804
Epoch  10 | Train Loss: 58.22225522249937
=========> Train acc: 0.964 | Test acc: 0.801
Epoch  11 | Train Loss: 51

학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.

In [5]:

import torch
from torch import nn

softmax1 = nn.Softmax()  #
softmax2 = nn.Softmax(dim=-1)  # 마지막 차원에 적용
x = torch.tensor([[[1.0, 2.0, 3.0],
                   [4.0, 5.0, 6.0]],

                  [[2.0, 4.0, 1.0],
                   [5.0, 3.0, 1.0]]])

print(x.shape)
print(x)
output1 = softmax1(x)
output2 = softmax2(x)
print(x)
print(output1)
print(output2)


torch.Size([2, 2, 3])
tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[2., 4., 1.],
         [5., 3., 1.]]])
tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[2., 4., 1.],
         [5., 3., 1.]]])
tensor([[[0.2689, 0.1192, 0.8808],
         [0.2689, 0.8808, 0.9933]],

        [[0.7311, 0.8808, 0.1192],
         [0.7311, 0.1192, 0.0067]]])
tensor([[[0.0900, 0.2447, 0.6652],
         [0.0900, 0.2447, 0.6652]],

        [[0.1142, 0.8438, 0.0420],
         [0.8668, 0.1173, 0.0159]]])


/opt/anaconda3/envs/hanghae99_plusAI_2/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
